<a href="https://colab.research.google.com/github/udaykiran2495/rag/blob/master/berskhire_reports_indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 13.8 MB/s eta 0:00:00


In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.9 MB/s eta 0:00:00


In [3]:
import requests
import os

In [4]:
os.makedirs("berkshire_pdfs", exist_ok=True)

In [5]:
years = list(range(1998, 2025))  # 2025 is exclusive, so this goes up to 2024

base_url = "https://www.berkshirehathaway.com/"
for year in years:
    report = str(year) + "ar"
    url = base_url + report + "/" + report + ".pdf"
    print(f"Downloading report for year {year} ...")

    response = requests.get(url)
    with open(f"berkshire_pdfs/{report}.pdf", "wb") as f:
        f.write(response.content)

print("Downloaded reports")


Downloaded reports


In [6]:
import pandas as pd
from pinecone import Pinecone

pc = Pinecone(
    api_key="pcsk_cCQ8i_EovFgsagt6XKD5QCHqTZuQMAnQUPAfNygTEhCAvTo7b5VubkghF59HMLucMMNTg",
    environment="us-east-1",
)


In [7]:
from transformers import pipeline, AutoTokenizer


HFTOKEN needed


In [8]:
# Initialize embedding pipeline (using a general-purpose model)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = pipeline("feature-extraction", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
# prompt: get file paths from the os

file_paths = [os.path.join("berkshire_pdfs", f) for f in os.listdir("berkshire_pdfs") if f.endswith(".pdf")]
print(file_paths)

['berkshire_pdfs/2024ar.pdf', 'berkshire_pdfs/2002ar.pdf', 'berkshire_pdfs/1999ar.pdf', 'berkshire_pdfs/2008ar.pdf', 'berkshire_pdfs/2020ar.pdf', 'berkshire_pdfs/2018ar.pdf', 'berkshire_pdfs/2017ar.pdf', 'berkshire_pdfs/2015ar.pdf', 'berkshire_pdfs/2011ar.pdf', 'berkshire_pdfs/2016ar.pdf', 'berkshire_pdfs/2004ar.pdf', 'berkshire_pdfs/2009ar.pdf', 'berkshire_pdfs/2023ar.pdf', 'berkshire_pdfs/2021ar.pdf', 'berkshire_pdfs/1998ar.pdf', 'berkshire_pdfs/2005ar.pdf', 'berkshire_pdfs/2019ar.pdf', 'berkshire_pdfs/2003ar.pdf', 'berkshire_pdfs/2013ar.pdf', 'berkshire_pdfs/2012ar.pdf', 'berkshire_pdfs/2022ar.pdf', 'berkshire_pdfs/2000ar.pdf', 'berkshire_pdfs/2010ar.pdf', 'berkshire_pdfs/2014ar.pdf', 'berkshire_pdfs/2001ar.pdf', 'berkshire_pdfs/2007ar.pdf', 'berkshire_pdfs/2006ar.pdf']


In [10]:
import pdfplumber
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text



In [11]:


# prompt: iterate through each of the files in file_paths
max_tokens = 384 # Emedding vector size
token_chunks = []

for pdf_path in file_paths:
    print(f"Processing file: {pdf_path}")

    text = extract_text_from_pdf(pdf_path)
    #chunks = [chunk for chunk in text.split('\n\n') if chunk.strip()]
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]



    for i in range(0, len(tokens), max_tokens):
        sub_tokens = tokens[i:i + max_tokens]
        sub_text = tokenizer.decode(sub_tokens, skip_special_tokens=True)
        if sub_text.strip():
            token_chunks.append(sub_text)

    print('Chunks so far:' + str(len(token_chunks)))

print('Total chunks created: ' + str(len(token_chunks)))




Processing file: berkshire_pdfs/2024ar.pdf


Token indices sequence length is longer than the specified maximum sequence length for this model (149186 > 512). Running this sequence through the model will result in indexing errors


Chunks so far:389
Processing file: berkshire_pdfs/2002ar.pdf
Chunks so far:641
Processing file: berkshire_pdfs/1999ar.pdf
Chunks so far:831
Processing file: berkshire_pdfs/2008ar.pdf
Chunks so far:1151
Processing file: berkshire_pdfs/2020ar.pdf
Chunks so far:1473
Processing file: berkshire_pdfs/2018ar.pdf
Chunks so far:1894
Processing file: berkshire_pdfs/2017ar.pdf
Chunks so far:2343
Processing file: berkshire_pdfs/2015ar.pdf
Chunks so far:2724
Processing file: berkshire_pdfs/2011ar.pdf
Chunks so far:3077
Processing file: berkshire_pdfs/2016ar.pdf
Chunks so far:3513
Processing file: berkshire_pdfs/2004ar.pdf
Chunks so far:3804
Processing file: berkshire_pdfs/2009ar.pdf
Chunks so far:4128
Processing file: berkshire_pdfs/2023ar.pdf
Chunks so far:4528
Processing file: berkshire_pdfs/2021ar.pdf
Chunks so far:4898
Processing file: berkshire_pdfs/1998ar.pdf
Chunks so far:5078
Processing file: berkshire_pdfs/2005ar.pdf
Chunks so far:5380
Processing file: berkshire_pdfs/2019ar.pdf
Chunks so f

In [12]:
import numpy as np

vectors = []

i = 0
for chunk in token_chunks:
    output = embedder(chunk)
    token_vecs = np.array(output[0])  # shape: (tokens, emb_dim)
    chunk_vec = token_vecs.mean(axis=0)  # mean pooling over tokens
    vectors.append(chunk_vec)

    i = i+ 1
    if i % 50 == 0:
        print(f"Processed {i} chunks")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 50 chunks
Processed 100 chunks
Processed 150 chunks
Processed 200 chunks
Processed 250 chunks
Processed 300 chunks
Processed 350 chunks
Processed 400 chunks
Processed 450 chunks
Processed 500 chunks
Processed 550 chunks
Processed 600 chunks
Processed 650 chunks
Processed 700 chunks
Processed 750 chunks
Processed 800 chunks
Processed 850 chunks
Processed 900 chunks
Processed 950 chunks
Processed 1000 chunks
Processed 1050 chunks
Processed 1100 chunks
Processed 1150 chunks
Processed 1200 chunks
Processed 1250 chunks
Processed 1300 chunks
Processed 1350 chunks
Processed 1400 chunks
Processed 1450 chunks
Processed 1500 chunks
Processed 1550 chunks
Processed 1600 chunks
Processed 1650 chunks
Processed 1700 chunks
Processed 1750 chunks
Processed 1800 chunks
Processed 1850 chunks
Processed 1900 chunks
Processed 1950 chunks
Processed 2000 chunks
Processed 2050 chunks
Processed 2100 chunks
Processed 2150 chunks
Processed 2200 chunks
Processed 2250 chunks
Processed 2300 chunks
Processe

In [13]:
index_name = "berkshire-reports"
index = pc.Index(index_name)

In [14]:
for idx, vector in enumerate(vectors):
    # Create a unique ID for each vector
    vector_id = f"chunk_{idx}"

    # Prepare the metadata (optional)
    metadata = {
        "source": pdf_path,
        "text": token_chunks[idx]
    }

    # Upsert the vector into the index
    index.upsert(vectors=[(vector_id, vector.tolist(), metadata)])